In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import auc, multilabel_confusion_matrix,ConfusionMatrixDisplay,confusion_matrix,roc_curve
import matplotlib.pyplot as plt

from Model import Model

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#load csv into dataframes
crime_data = "San_Francisco.csv"
df_crime = pd.read_csv(crime_data)